<a href="https://colab.research.google.com/github/sargent-mg/ml-zoomcamp/blob/main/08-deep-learning/homework_w8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Zoomcamp Homework 8: CNNs

This notebook contains the solution for the Convolutional Neural Networks homework.

### 1. Setup and Data Preparation

First, we import the necessary libraries and set the seeds for reproducibility as specified in the homework.

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os

# Reproducibility settings
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Download the Dataset

In [2]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip -q data.zip

--2025-12-01 20:38:41--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-01T21%3A26%3A47Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-01T20%3A26%3A04Z&ske=2025-12-01T21%3A26%3A47Z&sks=b&skv=2018-11-09&sig=btyi7sLhcjgtacrWuA4WTzNQhq4ajTYaYi%2BC8xZ8Y3M%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDYyMzMyMSwibmJmIjoxNzY0NjIxNTIxLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

### 2. Model Definition

We define the Convolutional Neural Network structure as described:
* Input: `(3, 200, 200)`
* Conv2d: 32 filters, kernel (3,3), stride 1, padding 0, ReLU
* MaxPool2d: (2,2)
* Flatten
* Linear: 64 neurons, ReLU
* Output: 1 neuron

In [3]:
class HairClassifier(nn.Module):
    def __init__(self):
        super(HairClassifier, self).__init__()
        # Conv Layer: 3 input channels, 32 output channels, 3x3 kernel
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0, stride=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Flattening
        self.flatten = nn.Flatten()

        # Calculate size after Conv + Pool
        # Input: 200x200
        # Conv (3x3, no pad): 200 - 3 + 1 = 198
        # Pool (2x2): 198 / 2 = 99
        # Feature map size: 32 * 99 * 99
        self.fc_input_size = 32 * 99 * 99

        self.fc1 = nn.Linear(self.fc_input_size, 64)
        self.fc2 = nn.Linear(64, 1) # Binary classification output

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.flatten(x)

        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        # Note: No Sigmoid here because we will use BCEWithLogitsLoss
        # The provided training code applies torch.sigmoid(outputs) manually for accuracy calc
        return x

### Question 1: Loss Function
Since we are doing binary classification and the provided training loop applies `sigmoid` to the outputs manually (`torch.sigmoid(outputs) > 0.5`), the model should output **logits**.

Therefore, the correct loss function is **`nn.BCEWithLogitsLoss()`**.

### Question 2: Number of Parameters

In [4]:
model = HairClassifier().to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


In [5]:
%pip install torchsummary

from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
           Flatten-4               [-1, 313632]               0
            Linear-5                   [-1, 64]      20,072,512
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 23.93
Params size (MB): 76.57
Estimated Total Size (MB): 100.96
----------------------------------------------------------------


### 3. Training Preparation

We define the transforms and create the DataLoaders.

In [6]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Dataset Setup
train_dataset = ImageFolder('./data/train', transform=train_transforms)
validation_dataset = ImageFolder('./data/test', transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

# Optimizer and Loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()

### 4. Training Loop (First Run)

Running for 10 epochs as requested.

In [7]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6665, Acc: 0.6112, Val Loss: 0.6511, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5702, Acc: 0.6787, Val Loss: 0.6332, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5207, Acc: 0.7350, Val Loss: 0.6143, Val Acc: 0.6766
Epoch 4/10, Loss: 0.4773, Acc: 0.7600, Val Loss: 0.6049, Val Acc: 0.6617
Epoch 5/10, Loss: 0.4606, Acc: 0.7550, Val Loss: 0.7307, Val Acc: 0.5672
Epoch 6/10, Loss: 0.3954, Acc: 0.8275, Val Loss: 0.6412, Val Acc: 0.6866
Epoch 7/10, Loss: 0.2844, Acc: 0.8838, Val Loss: 0.8307, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2885, Acc: 0.8788, Val Loss: 0.7052, Val Acc: 0.7114
Epoch 9/10, Loss: 0.1882, Acc: 0.9313, Val Loss: 0.9275, Val Acc: 0.6866
Epoch 10/10, Loss: 0.2585, Acc: 0.8912, Val Loss: 0.8158, Val Acc: 0.6915


### Question 3: Median Training Accuracy

In [8]:
median_acc = np.median(history['acc'])
print(f"Median Training Accuracy: {median_acc:.4f}")

Median Training Accuracy: 0.7937


### Question 4: Standard Deviation of Training Loss

In [9]:
std_loss = np.std(history['loss'])
print(f"Std Training Loss: {std_loss:.4f}")

Std Training Loss: 0.1462


### 5. Data Augmentation

We now add the requested augmentations to the training set and continue training the **same** model for 10 more epochs.

In [10]:
aug_train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)), # Resize is needed after crop to ensure consistency if crop varies
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Update the training dataset with new transforms
train_dataset_aug = ImageFolder('./data/train', transform=aug_train_transforms)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)

print("Continuing training with augmentations...")

# We continue using 'history' to append new results
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    # Use the new augmented loader
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+11}/{num_epochs*2}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Continuing training with augmentations...
Epoch 11/20, Loss: 0.6307, Acc: 0.6600, Val Loss: 0.5963, Val Acc: 0.7114
Epoch 12/20, Loss: 0.5801, Acc: 0.6750, Val Loss: 0.5968, Val Acc: 0.7214
Epoch 13/20, Loss: 0.5374, Acc: 0.7175, Val Loss: 0.5870, Val Acc: 0.7015
Epoch 14/20, Loss: 0.5502, Acc: 0.7125, Val Loss: 0.5695, Val Acc: 0.7264
Epoch 15/20, Loss: 0.5250, Acc: 0.7412, Val Loss: 0.6758, Val Acc: 0.6816
Epoch 16/20, Loss: 0.5141, Acc: 0.7612, Val Loss: 0.5764, Val Acc: 0.7313
Epoch 17/20, Loss: 0.5217, Acc: 0.7362, Val Loss: 0.5756, Val Acc: 0.6866
Epoch 18/20, Loss: 0.4888, Acc: 0.7588, Val Loss: 0.5724, Val Acc: 0.7214
Epoch 19/20, Loss: 0.4989, Acc: 0.7475, Val Loss: 0.5452, Val Acc: 0.7313
Epoch 20/20, Loss: 0.4663, Acc: 0.7688, Val Loss: 0.5619, Val Acc: 0.7164


### Question 5: Mean Test Loss (Augmented)
Calculate mean of test loss for all epochs trained with augmentations (epochs 11-20).

In [11]:
# The history list now contains 20 items. Indices 10 to 19 correspond to the augmented training.
aug_val_losses = history['val_loss'][10:]
mean_aug_val_loss = np.mean(aug_val_losses)
print(f"Mean Val Loss (Augmented): {mean_aug_val_loss:.4f}")

Mean Val Loss (Augmented): 0.5857


### Question 6: Average Test Accuracy (Last 5 Epochs)
Calculate average test accuracy for the last 5 epochs (epochs 16-20).

In [12]:
last5_val_acc = history['val_acc'][-5:]
avg_last5_val_acc = np.mean(last5_val_acc)
print(f"Avg Val Acc (Last 5): {avg_last5_val_acc:.4f}")

Avg Val Acc (Last 5): 0.7174
